# Stem Counting in Intermediate wheatgrass
Two different programs have been used for stem (circle) detection: **OpenCV** and **Skimage**. Both rely on the Hough transform to detect circles. 

Each program was tested with an RGB image as well as a L*a*b* image with $0 \leq a* < 125$ in an integer range [0...255] as in ImageJ, or $-127 \leq a* < 0$ in an integer range [-127...127] as in the CIELAB scale.

## Dependencies

In [ ]:
# CustomFunction is my own package with functions I have created to facilitate my work. It is not totally necessary for this purpose.
from CustomFunctions import *

import time
# start_time = time.time()

import os

import glob

import numpy as np

import cv2 as cv

import matplotlib.pyplot as plt

from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter, circle_perimeter_aa
from skimage.draw import disk, set_color
from skimage.util import img_as_ubyte

from scipy import ndimage, misc
from PIL import Image

import pandas as pd

import pathlib

# Change the "%matplotlib inline" figure resolution on the notebook if desired
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 600

# Make sure Jupyter Notebook shows all outputs from the same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## Test image

In [ ]:
%%time
mypath = r'./Images/Stems/Testing'
Images = glob.glob(mypath + '/**/*.JPG', recursive=True)
img0 = plt.imread(Images[0])
plt.imshow(img0)
print("Image ", Images[0])

## Using OpenCV
The function will detect circles on either an rgb or a Lab image. 
For an rgb image (LAB=False), a Canny edge detector will be used to identify the stems and the output will be passed to the Hough circle function. 
For a Lab image (LAB=True), Hough will be executed on the thresholded Lab given the minLAB and maxLAB values.

In [ ]:
def DetectStemsCV(img, LAB=True, minLAB=[140,0,0], maxLAB=[255,125,255], FilledCircles=False):
    
    # The function will detect circles on either an rgb or a Lab image.
    # For an rgb image (LAB=False), a Canny edge detector will be passed to the Hough circle.
    # For a Lab image (LAB=True, and min and max values given as lists), Hough will be executed on the thresholded Lab given the minLAB and maxLAB values.
    
    # Read image
    if isinstance(img, str) == True:
        img = plt.imread(img)
    else: 
        img = img
    
    output = img.copy()
    
    if LAB == True and minLAB is not None and maxLAB is not None:
        LAB = cv.cvtColor(img, cv.COLOR_BGR2LAB)
        minLAB = np.array(minLAB)
        maxLAB = np.array(maxLAB)
        maskLAB = cv.inRange(LAB, minLAB, maxLAB)
        LAB = cv.bitwise_and(img, img, mask = maskLAB)
        gray = cv.cvtColor(LAB, cv.COLOR_BGR2GRAY)
        gray = cv.medianBlur(gray, 5)
        circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, dp=1, minDist=40,
                              param1=60, param2=28, minRadius=1, maxRadius=45)
    else:
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        gray = cv.medianBlur(gray, 5)
        edges = cv.Canny(gray,50,150)
        circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 1, minDist=40,
                              param1=60, param2=28, minRadius=20, maxRadius=45)
    
    detected_circles = np.uint16(np.around(circles)) # list of circle parameters (x, y, radius)
    
    
    if FilledCircles == True:
        for (center_x, center_y, radius) in detected_circles[0, :]:
            circy, circx = disk((center_x, center_y), radius,
                                            shape=img.shape)
            set_color(output, (circx, circy), [255, 0, 0], alpha=0.5)
    else:
        for (x, y ,r) in detected_circles[0, :]:
            cv.circle(output, (x, y), r, (255, 0, 0), 2)
        

    return output, len(detected_circles[0])


Testing on RGB

In [ ]:
%%time

Stems, nStems = DetectStemsCV(img0, LAB=False, minLAB=[0,0,0], maxLAB=[255,125,255], FilledCircles=False)
plt.imshow(Stems)
print("\n", nStems, " stems identified.", "\n")

Testing on LAB with a* < 125

In [ ]:
%%time

Stems, nStems = DetectStemsCV(img0, LAB=True, minLAB=[0,0,0], maxLAB=[255,125,255], FilledCircles=False)
plt.imshow(Stems)
print("\n", nStems, " stems identified.", "\n")

## Using Skimage
Unlike with the OpenCV-based function, here we use a Canny detector on both the rgb and the thresholded Lab. 

We can currently change only the upper bound of the a* channel without using OpenCV. We could add parameters to change the other channels if needed but it seems to be more computationally intensive with Skimage, so we may consider using openCV ucntions such as `inRange`.

In [ ]:
def DetectStems(img, MedianSize=7, CannySigma=1, CannyLow=50, CannyHigh=150, LAB=True, minLAB=[0,0,0], maxLAB=[255,125,255]):
    
    # Read image
    if isinstance(img, str) == True:
        img = plt.imread(img)
    else: 
        img = img
    
    output = img.copy()
    
    # Detect radii
    hough_radii = np.arange(20, 50, 1)
    
    if LAB == True and minLAB is not None and maxLAB is not None:
        LAB = cv.cvtColor(img, cv.COLOR_BGR2LAB)
        minLAB = np.array(minLAB)
        maxLAB = np.array(maxLAB)
        maskLAB = cv.inRange(LAB, minLAB, maxLAB)
        LAB = cv.bitwise_and(img, img, mask = maskLAB)
        gray0 = LAB @ [0.2126, 0.7152, 0.0722]
        gray0 = ndimage.median_filter(gray0, size=MedianSize)
        hough_res = hough_circle(gray0, hough_radii)
    else: 
        gray0 = img @ [0.2126, 0.7152, 0.0722]
        gray0 = ndimage.median_filter(gray0, size=MedianSize)
        edges = canny(gray0, sigma = CannySigma, low_threshold=CannyLow, high_threshold=CannyHigh)
        hough_res = hough_circle(edges, hough_radii)
    
    # Get peaks
    accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, min_xdistance=30, min_ydistance=30, threshold=0.34)
    
    for center_y, center_x, radius in zip(cy, cx, radii):
        circy, circx = disk((center_y, center_x), radius, shape=output.shape)
        set_color(output, (circy, circx), [255, 0, 0], alpha=0.5)


    return output, len(radii)
    
    


Testing on RGB

In [ ]:
%%time

Stems, nStems = DetectStems(img0, MedianSize = 5, CannySigma = 1, CannyLow=10, CannyHigh=80, LAB=False)
plt.imshow(Stems)
print("\n", nStems, " stems identified.", "\n")


Testing on LAB with a* < 0

In [ ]:
%%time

Stems, nStems = DetectStems(img0, MedianSize = 5, CannySigma = 1, CannyLow=5, CannyHigh=150, LAB=True, minLAB=[100,0,0], maxLAB=[255,125,255])
plt.imshow(Stems)
print("\n", nStems, " stems identified.", "\n")

# Executing OpenCV's on entire images' folder

In [ ]:
# Define image folder
mypath = r'./Images/Stems/Original'
Images = glob.glob(mypath + '/*.JPG', recursive=False)
Images

In [ ]:
%%time

Stems_data = pd.DataFrame()

CountColname = "LABa0-140"

for img in Images:
    
    # Read the image
    img0 = iio.imread(img)
#     img0 = CropStems(img)
    
    # Create output image and get number of detected stems
    Stems, nStems = DetectStemsCV(img0, LAB=True, minLAB=[0,0,0], maxLAB=[255,140,255], FilledCircles=False)
    
    # Create path to nested subdirectory for output image
    FullPath = img.split("\\")[:1]
    FullPath = '\\'.join([str(i) for i in FullPath])
    FullPath = FullPath + "\\DetectedStems\\"
    output_name = img.split("\\")[-1]
    barcode = output_name.replace(".JPG", "")
    barcode = barcode.split("_")[-1]
    output_name = output_name.replace(".JPG", "_DC.JPG")
    
    # Check if folder exist, create otherwise
    path = pathlib.Path(FullPath)
    path.mkdir(parents=True, exist_ok=True)
    
    # Define full filename and save image
    output_name = FullPath + output_name
    imgWritten = cv.imwrite(output_name, cv.cvtColor(Stems, cv.COLOR_RGB2BGR))
    
    if imgWritten:
          print("Image saved as ", output_name)
    
    img_data = pd.DataFrame([(int(barcode), nStems)], columns=["barcode",CountColname])
    Stems_data = Stems_data.append(img_data)
    

## Datasets

In [ ]:
Stems_data
Stems_data2 = Stems_data.reset_index()
Stems_data2
# data_name = FullPath + "Stems_data.csv"
# Stems_data.to_csv(data_name, index=False)

In [ ]:
# Read manual count file
MC = pd.read_excel(".\\stem_dat.xlsx", engine='openpyxl')
MC
MC['barcode'] = MC['barcode'].apply(lambda x: x if pd.isnull(x) else str(int(x)))
MC

In [ ]:
# Merge manual and automatic count
# df1 = pd.merge(Stems_data2, MC, on = ['barcode'])
df1 = pd.concat([Stems_data2, MC], axis=0, ignore_index=True, sort=False)

# df1 = Stems_data.join(MC, how='left', lsuffix='_left', rsuffix='_right')
# df1

# df1.dropna(subset = ["manual_tiller_ct"], inplace=True)
# df1
data_name = FullPath + "df1.csv"
df1.to_csv(data_name, index=False)


In [ ]:
df1.columns
df2 = df1[['index', 'barcode', 'LABa0-140','manual_tiller_ct', 'rep']]
df2

## Plotting

In [ ]:
# df3 = df2.dropna(subset = ["rep"], inplace=True)
# df3

# Change the "%matplotlib inline" figure resolution on the notebook if desired
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 200

import plotnine as p9

In [ ]:
df3 = df2[['barcode','rep']]
p = p9.ggplot(data=df3, mapping=p9.aes(x='rep', y='barcode'))
# Draw the plot
p + p9.geom_point()

In [ ]:
df = df2[[CountColname,'manual_tiller_ct']]
df.corr()
      dogs  cats
dogs   1.0   0.3
cats   0.3   1.0

# Ignore anything below here

In [ ]:
blurry = filters.gaussian(edge_sobel, sigma = 5)
plt.imshow(blurry)

In [ ]:
# sk = skeletonize(img1 > 20)
# plt.imshow(sk, cmap='gray')

In [ ]:
# img = cv.imread('messi5.jpg',0)
blur = cv.medianBlur(img0,7)
edges = cv.Canny(blur,100,200)
plt.imshow(edges)
# blur = cv.medianBlur(edges,3)
# plt.imshow(blur)

In [ ]:
def DetectStemsCV(img, LAB=True, minLAB=[140,0,0], maxLAB=[255,125,255], FilledCircles=False):
    
    # The function will detect circles on either an rgb or a Lab image.
    # For an rgb image (LAB=False), a Canny edge detector will be passed to the Hough circle.
    # For a Lab image (LAB=True, and min and max values given as lists), Hough will be executed on the thresholded Lab given the minLAB and maxLAB values.
    
    output = img.copy()
    
    if LAB == True and minLAB is not None and maxLAB is not None:
        LAB = cv.cvtColor(img, cv.COLOR_BGR2LAB)
        minLAB = np.array(minLAB)
        maxLAB = np.array(maxLAB)
        maskLAB = cv.inRange(LAB, minLAB, maxLAB)
        LAB = cv.bitwise_and(img, img, mask = maskLAB)
        gray = cv.cvtColor(LAB, cv.COLOR_BGR2GRAY)
        gray = cv.medianBlur(gray, 5)
        circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, dp=1, minDist=50,
                              param1=60, param2=28, minRadius=20, maxRadius=45)
    else:
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        gray = cv.medianBlur(gray, 5)
        edges = cv.Canny(gray,50,150)
        circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 1, minDist=50,
                              param1=60, param2=28, minRadius=20, maxRadius=45)
    
    detected_circles = np.uint16(np.around(circles)) # list of circle parameters (x, y, radius)
    
    
    if FilledCircles == True:
        for (center_x, center_y, radius) in detected_circles[0, :]:
            circy, circx = disk((center_x, center_y), radius,
                                            shape=img.shape)
            set_color(output, (circx, circy), [255, 0, 0], alpha=0.5)
    else:
        for (x, y ,r) in detected_circles[0, :]:
            cv.circle(output, (x, y), r, (255, 0, 0), 3)
        

    return output, detected_circles


In [ ]:
test_hc, test_dc = DetectStemsCV(img0, LAB=True, minLAB=[0,0,0], maxLAB=[255,125,255], FilledCircles=False)
plt.imshow(test_hc)
len(test_dc[0])

In [ ]:
circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 1, minDist=50,
                              param1=60, param2=28, minRadius=40, maxRadius=60)
# len(circles[0])
detected_circles = np.uint16(np.around(circles))
for (center_y, center_x, radius) in detected_circles[0, :]:
    print(center_y, center_x, radius)


# for center_y, center_x, radius in zip(cy, cx, radii):
#         circy, circx = disk((center_y, center_x), radius,
#                                         shape=image.shape)
#         set_color(image, (circy, circx), [255, 0, 0], alpha=0.5)
# #         image[circy, circx] = (255,0,0)

#     return image, len(radii)

In [ ]:
lab_image = cv.cvtColor(img0, cv.COLOR_BGR2LAB)
l_channel,a_channel,b_channel = cv.split(lab_image)
plt.imshow(lab_image)

In [ ]:
a_channel.max()

In [ ]:
# lab = cv.cvtColor(img0, cv.COLOR_BGR2LAB)\thresh = 40
lab = cv.cvtColor(img0, cv.COLOR_BGR2LAB)
# plt.imshow(lab[:,:,1])
# hsv = [65, 229, 158]
minLAB = np.array([140, 0, 0])
maxLAB = np.array([255, 125, 255])

maskLAB = cv.inRange(lab, minLAB, maxLAB)
resultLAB = cv.bitwise_and(img0, img0, mask = maskLAB)
plt.imshow(resultLAB)
# minHSV.shape

In [ ]:
gray = cv.cvtColor(resultLAB, cv.COLOR_BGR2GRAY)
plt.imshow(gray)

In [ ]:
gray = cv.cvtColor(img0, cv.COLOR_BGR2GRAY)
plt.imshow(gray, cmap='gray')

In [ ]:
ComparePlots(1,2, [test_hc, img0])

In [ ]:
len(test_dc[0])

In [ ]:
test_dc[:, 0, 0]

In [ ]:
test_hc, test_dc = hough_circle(img0, 70, 40)
bw0 = test_hc[:, :, 0] == 255
plt.imshow(bw0)

In [ ]:
edges = canny(img1,low_threshold=50, high_threshold=150)
plt.imshow(edges)

In [ ]:
# Detect two radii
hough_radii = np.arange(20, 35, 1)
hough_res = hough_circle(edges, hough_radii)

In [ ]:
hough_res.shape

In [ ]:
# hough_radii
# hough_res

# Select the most prominent 3 circles
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, min_xdistance=40, min_ydistance=40)

In [ ]:
len(radii)

In [ ]:
# # Draw them
# fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4))
# image = color.gray2rgb(img1)
# for center_y, center_x, radius in zip(cy, cx, radii):
#     circy, circx = disk((center_y, center_x), radius,
#                                     shape=image.shape)
    
#     image[circy, circx] = (255,0,0)

# ax.imshow(image, cmap=plt.cm.gray)
# plt.show()

# # plt.imshow(image)


# Draw them
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4))
image = color.gray2rgb(img1)
for center_y, center_x, radius in zip(cy, cx, radii):
    circy, circx = disk((center_y, center_x), radius,
                                    shape=image.shape)
    set_color(image, (circy, circx), [255, 0, 0], alpha=0.5)
    
#     image[circy, circx] = (255,0,0)

ax.imshow(image, cmap=plt.cm.gray)
plt.show()

# >>> from skimage.draw import circle_perimeter_aa
# >>> img = np.zeros((10, 10), dtype=np.uint8)
# >>> rr, cc, val = circle_perimeter_aa(4, 4, 3)
# >>> img[rr, cc] = val * 255
# >>> img
# array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
#        [  0,   0,  60, 211, 255, 211,  60,   0,   0,   0],
#        [  0,  60, 194,  43,   0,  43, 194,  60,   0,   0],
#        [  0, 211,  43,   0,   0,   0,  43, 211,   0,   0],
#        [  0, 255,   0,   0,   0,   0,   0, 255,   0,   0],
#        [  0, 211,  43,   0,   0,   0,  43, 211,   0,   0],
#        [  0,  60, 194,  43,   0,  43, 194,  60,   0,   0],
#        [  0,   0,  60, 211, 255, 211,  60,   0,   0,   0],
#        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
#        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0]], dtype=uint8)

# >>> from skimage import data, draw
# >>> image = data.chelsea()
# >>> rr, cc, val = draw.circle_perimeter_aa(r=100, c=100, radius=75)
# >>> draw.set_color(image, (rr, cc), [1, 0, 0], alpha=val)

In [ ]:
len(radii)

In [ ]:
ComparePlots(1,2, [test_hc, img0])

In [ ]:
len(radii)

In [ ]:
def hough_circle_cv(img, min_dist, max_radius):
    output = img.copy()
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # img2 = np.floor(img)
    # img2 = img2.astype(int)
    gray = cv.medianBlur(gray, 5)
    edges = cv.Canny(gray,50,150)
    circles = cv.HoughCircles(edges, cv.HOUGH_GRADIENT, 1, min_dist,
                              param1=50, param2=30, minRadius=0, maxRadius=max_radius)
    detected_circles = np.uint16(np.around(circles)) # its a list of circle parameters (x, y ,radius)
    for (x, y ,r) in detected_circles[0, :]:
        cv.circle(output, (x, y), r, (255, 0, 0), -1)
        # cv.circle(output, (x, y), 0, (0, 255, 0), 3)
        
    return output, detected_circles # output is the orig image with cirlcles drawn on it


In [ ]:
def DetectStems(ImagePath, MedianSize = 7, CannySigma = 1, CannyLow = 50, CannyHigh = 150):
    
#     edges = canny(img1,low_threshold=50, high_threshold=150)
#     plt.imshow(edges)
#     img0 = plt.imread(ImagePath)
#     img1 = CropStems(ImagePath)
#     img = Image.open('image.png').convert('LA')
    gray0 = img0 @ [0.2126, 0.7152, 0.0722]
#     gray0 = Image.fromarray(img0).convert('L')
#     gray0 = np.asarray(gray0)
    gray0 = ndimage.median_filter(gray0, size=MedianSize)
    edges = canny(gray0, sigma = CannySigma, low_threshold=CannyLow, high_threshold=CannyHigh)
    
    # Detect radii
    hough_radii = np.arange(20, 50, 1)
    hough_res = hough_circle(edges, hough_radii)
    
    # Get peaks
    accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, min_xdistance=40, min_ydistance=40, threshold=0.3)
    
    
    # Draw them
#     fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4))
    image = img0.copy()
    for center_y, center_x, radius in zip(cy, cx, radii):
        circy, circx = disk((center_y, center_x), radius,
                                        shape=image.shape)
        set_color(image, (circy, circx), [255, 0, 0], alpha=0.5)
#         image[circy, circx] = (255,0,0)

    return image, len(radii)
    
    


In [ ]:
stems_1, nStems = DetectStems(img0, MedianSize = 5, CannySigma = 0.8, CannyLow=10, CannyHigh=80)
# stems_1, nStems = DetectStems(rgb, MedianSize = 5, CannySigma = 0.8, CannyLow=10, CannyHigh=80)
plt.imshow(stems_1)
nStems


In [ ]:
stems_0, nStems = DetectStems(Images[0], MedianSize = 5, CannySigma = 1, CannyLow=10, CannyHigh=80)
plt.imshow(stems_0)
nStems


In [ ]:
plt.imshow(img0)

In [ ]:
gray0 = img0 @ [0.2126, 0.7152, 0.0722] 
gray0 = ndimage.median_filter(gray0, size=3)
# edges = canny(gray0, low_threshold=50, high_threshold=150)

In [ ]:
edges = canny(gray0, sigma = 0.5, low_threshold=50, high_threshold=150)
plt.imshow(edges, cmap = 'gray')

In [ ]:
# nvcc --version

In [ ]:
# import cupy as cp
# from skimage import data
# from cucim.skimage.feature import canny
# import numpy as np
# import matplotlib.pyplot as plt

# # from skimage import data, color
# # from skimage.transform import hough_circle, hough_circle_peaks
# from skimage.feature import canny
# # from skimage.draw import circle_perimeter, circle_perimeter_aa
# # from skimage.draw import disk, set_color
# # from skimage.util import img_as_ubyte

# # # Change the "%matplotlib inline" figure resolution on the notebook
# # import matplotlib as mpl
# # mpl.rcParams['figure.dpi']= 600

In [ ]:
def DetectStems(ImagePath, MedianSize = 7, CannySigma = 1, CannyLow = 50, CannyHigh = 150):
    
#     edges = canny(img1,low_threshold=50, high_threshold=150)
#     plt.imshow(edges)
    img0 = plt.imread(ImagePath)
    
    # Get Lab values
    Lab = color.rgb2lab(img0)
#     L = Lab[:,:,0]
#     a = Lab[:,:,1] < -3
#     b = Lab[:,:,2] > 18
#     Lab = color.rgb2lab(img0)
    a = Lab[:,:,1] < 0
    # a2 = a @ Lab[:,:,1]
    a2 = np.where(a[..., None], Lab, 0)
    rgb = color.lab2rgb(a2)
#     mask = Lab[:,:,0] > 60
    # Apply mask
#     Lab1 = np.where(mask[..., None], Lab, 0)
#     rgb = color.lab2rgb(Lab1)
#     img1 = CropStems(ImagePath)
#     img = Image.open('image.png').convert('LA')
    gray0 = rgb @ [0.2126, 0.7152, 0.0722]
#     gray0 = Image.fromarray(img0).convert('L')
#     gray0 = np.asarray(gray0)
#     gray0 = ndimage.median_filter(gray0, size=MedianSize)
    edges = canny(gray0, sigma = CannySigma, low_threshold=CannyLow, high_threshold=CannyHigh)
#     edges = canny(gray0, sigma=2, low_threshold=0.4, high_threshold=0.6)
    
    # Detect radii
    hough_radii = np.arange(20, 40, 1)
    hough_res = hough_circle(edges, hough_radii)
    
    # Get peaks
    accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, min_xdistance=40, min_ydistance=40, threshold=0.33)
    
    
    # Draw them
#     fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4))
    image = img0.copy()
    for center_y, center_x, radius in zip(cy, cx, radii):
        circy, circx = disk((center_y, center_x), radius,
                                        shape=image.shape)
        set_color(image, (circy, circx), [255, 0, 0], alpha=0.5)
#         image[circy, circx] = (255,0,0)

    return image, len(radii)
    
    


In [ ]:
stems_1, nStems = DetectStems(Images[0], MedianSize = 5, CannySigma = 0.6, CannyLow=0.58, CannyHigh=0.6)
plt.imshow(stems_1)
nStems

In [ ]:
stems_1, nStems = DetectStems(Images[0], MedianSize = 5, CannySigma = 0.6, CannyLow=0.2, CannyHigh=0.4)
plt.imshow(stems_1)
nStems
# This one detected 230, sort of accurate. Thresholding Lab based on L>60

In [ ]:
# rgb = plt.imread(img0)
# plt.imshow(img0)
# Get Lab values
Lab = color.rgb2lab(img0)
# mask = Lab[:,:,0] > 60
L = Lab[:,:,0]

a = Lab[:,:,1] @ Lab[:,:,1] < -3
b = Lab[:,:,2] @ Lab[:,:,2] > 18



Lab1 = np.dstack((L,a,b))  # stacks 3 h x w arrays -> h x w x 3
# rgb_uint8 = (np.dstack((L,a,b)) * 255.999) .astype(np.uint8)
# Lab1 = np.where(mask[..., None], Lab, 0)
rgb = color.lab2rgb(Lab1)
plt.imshow(rgb)
#     img1 = CropStems(ImagePath)
#     img = Image.open('image.png').convert('LA')
# gray0 = rgb @ [0.2126, 0.7152, 0.0722]




# Apply mask
# Lab1 = np.where(mask[..., None], Lab, 0)
# rgb = color.lab2rgb(Lab1)
# gray0 = rgb @ [0.2126, 0.7152, 0.0722]
# # gray0 = ndimage.median_filter(gray0, size=5)
# edges = canny(gray0)
# # plt.imshow(gray0, cmap='gray')
# plt.imshow(edges)

In [ ]:
Lab = color.rgb2lab(img0)
a = Lab[:,:,1] < -0
# a2 = a @ Lab[:,:,1]
a2 = np.where(a[..., None], Lab, 0)
rgb = color.lab2rgb(a2)
plt.imshow(rgb)
# plt.imshow(a2)
# a = Lab[:,:,1] @ (Lab[:,:,1] < -3)

In [ ]:
np.argwhere(image[:,:,0] > threshold)

In [ ]:
# a_mask = Lab[:,:,1] < -3
idx = Lab[:,:,2] > 18
Lab[idx,2] = 18
plt.imshow(Lab)
# a = np.dot(Lab[:,:,1], a_mask)
# a_mask.shape
# idx = image[:,:,0] > threshold
# image[idx,0] = threshold

In [ ]:
rgb = color.lab2rgb(Lab)
plt.imshow(rgb)

In [ ]:
edges = canny(gray0, sigma=2, low_threshold=0.4, high_threshold=0.6)
# plt.imshow(gray0, cmap='gray')
plt.imshow(edges)

In [ ]:
Lab[:,:,2].min()

In [ ]:
gray